In [12]:
import plotly.express as px
import pandas as pd
import os

In [ ]:
# Carregar os dados
df_aqe = pd.read_csv('/home/guilhermerc/Documentos/workspace/pyspark-playground/output/q3/aqe/aqe.grouped')
df_wo = pd.read_csv('/home/guilhermerc/Documentos/workspace/pyspark-playground/output/q3/wo/wo.grouped')
df_cbo = pd.read_csv('/home/guilhermerc/Documentos/workspace/pyspark-playground/output/q3/cbo/cbo.grouped')
df_cboaqe = pd.read_csv('/home/guilhermerc/Documentos/workspace/pyspark-playground/output/q3/cboaqe/cboaqe.grouped')

# Adicionar uma coluna para identificar o tipo de execução
df_aqe['execution_type'] = 'Com AQE'
df_wo['execution_type'] = 'Sem AQE'
df_cbo['execution_type'] = 'Com CBO'
df_cboaqe['execution_type'] = 'Com CBO+AQE'

# Combinar os dataframes
df_combined = pd.concat([df_aqe, df_wo, df_cbo,df_cboaqe])

In [14]:
# Importar as métricas relevantes
metricas_relevantes = [
    # Métricas de tempo de execução
    "elapsedTime",  # Tempo total de execução em ms
    "executorRunTime",  # Tempo total gasto pelos executores
    "executorCpuTime",  # Tempo de CPU utilizado pelos executores
    "jvmGCTime",  # Tempo gasto em coleta de lixo
    # Eficiência de recursos e gargalos
    "shuffleFetchWaitTime",  # Tempo esperando por dados de shuffle
    "shuffleWriteTime",  # Tempo gasto escrevendo dados de shuffle
    # Utilização de memória e spill
    "peakExecutionMemory",  # Pico de memória utilizada
    "diskBytesSpilled",  # Dados despejados em disco quando memória insuficiente
    "memoryBytesSpilled",  # Dados despejados em memória
    # Métricas de shuffle
    "shuffleTotalBytesRead",  # Bytes totais lidos durante shuffle
    "shuffleBytesWritten",  # Bytes escritos durante shuffle
    "shuffleRecordsRead",  # Registros lidos durante shuffle
    "shuffleRecordsWritten",  # Registros escritos durante shuffle
    # Métricas de entrada/saída
    "recordsRead",  # Registros lidos
    "bytesRead",  # Bytes lidos
    "recordsWritten",  # Registros escritos
    "bytesWritten",  # Bytes escritos
    # Paralelismo
    "numStages",  # Número de estágios
    "numTasks",  # Número de tarefas
    "avg_active_tasks",  # Média de tarefas ativas (paralelismo)
]

In [15]:
# Função para criar gráficos para cada métrica
def create_metric_chart(metric_name, df):
    # Definir unidades e títulos baseados na métrica
    if 'Time' in metric_name or metric_name == 'elapsedTime':
        unit = 'Tempo (ms)'
    elif 'Bytes' in metric_name or 'Memory' in metric_name:
        unit = 'Bytes'
    elif 'Records' in metric_name:
        unit = 'Registros'
    else:
        unit = 'Valor'
    
    title = f'Comparação de {metric_name} com e sem AQE'
    
    # Criar o gráfico de barras
    fig = px.bar(
        df, 
        x='execution_type', 
        y=metric_name,
        title=title,
        labels={'value': unit, 'execution_type': 'Tipo de Execução'},
        text_auto=True,
        color='execution_type',
        color_discrete_map={'Com AQE': 'blue', 'Sem AQE': 'red'}
    )
    
    # Melhorar o layout
    fig.update_layout(
        xaxis_title="",
        yaxis_title=unit,
        legend_title="Execução"
    )
    
    return fig

In [16]:
# Criar e exibir gráficos para cada métrica
for metric in metricas_relevantes:
    fig = create_metric_chart(metric, df_combined)
    fig.show()

In [17]:
# Criar um gráfico de barras agrupadas para métricas de tempo
time_metrics = ["elapsedTime", "executorRunTime", "executorCpuTime", "jvmGCTime"]
fig_time = px.bar(
    df_combined.melt(
        id_vars=['execution_type', 'query'],
        value_vars=time_metrics,
        var_name='metric',
        value_name='value'
    ),
    x='metric',
    y='value',
    color='execution_type',
    barmode='group',
    title='Comparação de Métricas de Tempo',
    labels={'value': 'Tempo (ms)', 'metric': 'Métrica', 'execution_type': 'Tipo de Execução'}
)
fig_time.show()

In [18]:
# Criar um gráfico de barras para métricas de paralelismo
parallelism_metrics = ["numStages", "numTasks", "avg_active_tasks"]
fig_parallelism = px.bar(
    df_combined.melt(
        id_vars=['execution_type', 'query'],
        value_vars=parallelism_metrics,
        var_name='metric',
        value_name='value'
    ),
    x='metric',
    y='value',
    color='execution_type',
    barmode='group',
    title='Comparação de Métricas de Paralelismo',
    labels={'value': 'Quantidade', 'metric': 'Métrica', 'execution_type': 'Tipo de Execução'}
)
fig_parallelism.show()

In [19]:
# Criar um gráfico de barras para métricas de tempo de shuffle
shuffle_time_metrics = ["shuffleFetchWaitTime", "shuffleWriteTime"]
fig_shuffle_time = px.bar(
    df_combined.melt(
        id_vars=['execution_type', 'query'],
        value_vars=shuffle_time_metrics,
        var_name='metric',
        value_name='value'
    ),
    x='metric',
    y='value',
    color='execution_type',
    barmode='group',
    title='Comparação de Métricas de Tempo de Shuffle',
    labels={'value': 'Tempo (ms)', 'metric': 'Métrica', 'execution_type': 'Tipo de Execução'}
)
fig_shuffle_time.show()

In [20]:
# Criar um gráfico de barras para métricas de dados de shuffle
shuffle_data_metrics = ["shuffleTotalBytesRead", "shuffleBytesWritten", "shuffleRecordsRead", "shuffleRecordsWritten"]
fig_shuffle_data = px.bar(
    df_combined.melt(
        id_vars=['execution_type', 'query'],
        value_vars=shuffle_data_metrics,
        var_name='metric',
        value_name='value'
    ),
    x='metric',
    y='value',
    color='execution_type',
    barmode='group',
    title='Comparação de Métricas de Dados de Shuffle',
    labels={'value': 'Quantidade', 'metric': 'Métrica', 'execution_type': 'Tipo de Execução'}
)
fig_shuffle_data.update_layout(yaxis_type="log")  # Escala logarítmica para melhor visualização
fig_shuffle_data.show()